# Introduction of Linear decision trees

In this jupyter notebook, I am going to discuss how Linear decision trees can be added to Gogeta

In [1]:
# import necessary libraries 
using EvoTrees
using JuMP
using Gurobi

The key difference is that Linear decision trees have a linear function at the leafs, while normal decision tree has a coefficient. Lest create a dataset to learn a simple function:
$$f(x) = \sum_{i=1}^5 x_i^3$$



In [2]:
data = rand(1000, 5) .- 0.5;
x_train = data[1:750, :];
y_train = vec(sum(map.(x->x^3, x_train), dims=2));

x_test = data[751:end, :];
y_test = vec(sum(map.(x->x^3, x_test), dims=2));

Now using `EvoTrees`, lets create forest of 2 trees 

In [3]:
config = EvoTreeRegressor(nrounds=1, max_depth=5);
evo_model = fit_evotree(config; x_train, y_train, verbosity=0);

The first tree is always bias tree

In [4]:
evo_model.trees[1]

EvoTrees.Tree{EvoTrees.MSE, 1}
 - feat: [0]
 - cond_bin: UInt8[0x00]
 - cond_float: Any[0.0]
 - gain: [0.0]
 - pred: Float32[-0.0028208059;;]
 - split: Bool[0]


These are predictions learned by general tree. With Linear decision tree, we would also have $R^n$ vector for each leaf in the tree 

In [5]:
leaves = findall(node -> evo_model.trees[2].split[node] == false && (node == 1 || evo_model.trees[2].split[floor(Int, node / 2)] == true), 1:length(evo_model.trees[2].split))

evo_model.trees[2].pred[leaves]

16-element Vector{Float32}:
 -0.013679789
 -0.004538178
 -0.009680798
 -0.0010463883
  0.0016119897
  0.010375644
  0.013378836
  0.02618761
 -0.0073071234
  0.009128554
 -0.0013803802
  0.012229258
  0.003046792
  0.011406319
  0.01262668
  0.02374924

If Linear decision trees to be introduced in Julia, I would expect them to have similar structure as `EvoTrees`

In [6]:
struct LTEModel
    n_trees::Int64
    n_feats::Int64
    n_leaves::Array{Int64}
    leaves::Array{Array}
    splits::Matrix{Any}
    splits_ordered::Array{Vector}
    n_splits::Array{Int64}
    a::Array{Array}
    b::Array{Array}
    split_nodes::Array{Array}
    child_leaves::Array{Array}
end

In [7]:
function extract_evotrees_info(evo_model; tree_limit=length(evo_model.trees))

    n_trees = tree_limit
    n_feats = length(evo_model.info[:fnames])

    n_leaves = Array{Int64}(undef, n_trees) # number of leaves on each tree
    leaves = Array{Array}(undef, n_trees) # ids of the leaves of each tree

    # Get number of leaves and ids of the leaves on each tree
    for tree in 1:n_trees
        leaves[tree] = findall(node -> evo_model.trees[tree].split[node] == false && (node == 1 || evo_model.trees[tree].split[floor(Int, node / 2)] == true), 1:length(evo_model.trees[tree].split))
        n_leaves[tree] = length(leaves[tree])
    end

    splits = Matrix{Any}(undef, n_trees, length(evo_model.trees[2].split)) # storing the feature number and splitpoint index for each split node
    splits_ordered = Array{Vector}(undef, n_feats) # splitpoints for each feature

    n_splits = zeros(Int64, n_feats)
    [splits_ordered[feat] = [] for feat in 1:n_feats]

    for tree in 1:n_trees
        for node in eachindex(evo_model.trees[tree].split)
            if evo_model.trees[tree].split[node] == true
                splits[tree, node] = [evo_model.trees[tree].feat[node], evo_model.trees[tree].cond_float[node]] # save feature and split value
                push!(splits_ordered[evo_model.trees[tree].feat[node]], evo_model.trees[tree].cond_float[node]) # push split value to splits_ordered
            end
        end
    end
    [unique!(sort!(splits_ordered[feat])) for feat in 1:n_feats] # sort splits_ordered and remove copies
    [n_splits[feat] = length(splits_ordered[feat]) for feat in 1:n_feats] # store number of split points

    for tree in 1:n_trees
        for node in eachindex(evo_model.trees[tree].split)
            if evo_model.trees[tree].split[node] == true
                
                feature::Int = splits[tree, node][1]
                value = splits[tree, node][2]

                splits[tree, node][2] = searchsortedfirst(splits_ordered[feature], value)

            end
        end
    end


    b = Array{Array}(undef, n_trees)
    [b[tree] = evo_model.trees[tree].pred for tree in 1:n_trees]
    # -------------------------------------------------------
    # in the Linear decision tree, there should be gradient in addition to coefficient
    a = Array{Array}(undef, n_trees)
    [a[tree] = zeros(length(evo_model.trees[tree].pred), n_feats) for tree in 1:n_trees]
    # -------------------------------------------------------
    
    split_nodes = Array{Array}(undef, n_trees)
    [split_nodes[tree] = evo_model.trees[tree].split for tree in 1:n_trees]

    return LTEModel(n_trees, n_feats, n_leaves, leaves, splits, splits_ordered, n_splits, a, b, split_nodes, Array{Array}(undef, n_trees))

end

extract_evotrees_info (generic function with 1 method)

In [8]:
LTE = extract_evotrees_info(evo_model; tree_limit=length(evo_model.trees))

LTEModel(2, 5, [1, 16], Array[[1], [16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31]], Any[#undef #undef … #undef #undef; [1.0, 1.0] [2.0, 3.0] … #undef #undef], Vector[Any[0.18588022654089661, 0.4205677422949932], Any[0.16318949692532522, 0.17613459442551815, 0.3142543416970481], Any[-0.25592039748035383, 0.34098227422575805], Any[-0.3145710646837453, 0.1660823910604102, 0.20651100394402205, 0.3289236763233086, 0.38702371930363905], Any[-0.30684761488771506, 0.36396254886116697, 0.43382178171959634]], [2, 3, 2, 5, 3], Array[[0.0 0.0 … 0.0 0.0], [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0]], Array[Float32[-0.0028208059;;], Float32[0.0 0.0 … 0.01262668 0.02374924]], Array[Bool[0], Bool[1, 1, 1, 1, 1, 1, 1, 1, 1, 1  …  0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], Array[#undef, #undef])

No changes in the function `init_TEModel!()`, the only change is that we work with `LTEModel` now.

In [9]:
function children(id::Int, leaf_dict::Dict, max::Int)

    result::Vector{Int} = []

    function inner(num)
        if num <= max
            leaf_index = get(leaf_dict, num, 0)
            if leaf_index != 0
                push!(result, leaf_index)
            end
            inner(num << 1)
            inner(num << 1 + 1)
        end
    end

    inner(id)

    return result

end

children (generic function with 1 method)

In [10]:
function init_LTEModel!(LTE::LTEModel)

    leaf_dict = Array{Dict}(undef, LTE.n_trees)
    [leaf_dict[tree] = Dict([(LTE.leaves[tree][leaf], leaf) for leaf in eachindex(LTE.leaves[tree])]) for tree in 1:LTE.n_trees]

    # pre-compute all children for all active nodes of all trees
    for tree in 1:LTE.n_trees
        
        nodes_with_split = findall(split -> split == true, LTE.split_nodes[tree])
        LTE.child_leaves[tree] = Array{Any}(undef, maximum(LTE.leaves[tree]))

        for node in [nodes_with_split; LTE.leaves[tree]]
            LTE.child_leaves[tree][node] = children(node, leaf_dict[tree], last(LTE.leaves[tree]))
        end
    end
end


init_LTEModel! (generic function with 1 method)

In [169]:
L_bounds = -0.5*ones(LTE.n_feats);
U_bounds = 0.5*ones(LTE.n_feats);
d_bounds = [-1, 1];

In [202]:
function calculate_bounds(model::JuMP.Model, LTE::LTEModel, leaf, tree, d_bounds)

    # Something is wrong there 
    # ------------------------------------

    @objective(model, Max,  LTE.a[tree][LTE.leaves[tree][leaf], :]'*model[:x]+LTE.b[tree][LTE.leaves[tree][leaf]] - model[:d][tree])

    optimize!(model)
    upper_bound = objective_value(model)

    set_objective_sense(model, MIN_SENSE)
    optimize!(model)

    lower_bound = objective_value(model)
    
    return [upper_bound, lower_bound]
    
end

calculate_bounds (generic function with 1 method)

In [209]:
function LTE_formulate!(model::JuMP.Model, LTE::LTEModel, U_bounds, L_bounds; bound_tightening = "quadratic", d_bounds=nothing)

    empty!(model)

 
    if bound_tightening == "output" @assert !isnothing(d_bounds) "Bounds for forest output must be provided." end

    init_LTEModel!(LTE)

    # (2.2.8)
    @variable(model, x[1:LTE.n_feats]);
    @constraint(model, [i = 1:LTE.n_feats], x[i] <= U_bounds[i]);
    @constraint(model, [i = 1:LTE.n_feats], x[i] >= L_bounds[i]);

    # (2.2.9)
    @variable(model, y[i = 1:LTE.n_feats, 1:LTE.n_splits[i]], Bin);

    # (2.2.10)
    @variable(model, z[tree = 1:LTE.n_trees, 1:LTE.n_leaves[tree]], Bin); 

    # (2.2.5)
    @constraint(model, [i = 1:LTE.n_feats, j = 1:(LTE.n_splits[i]-1)], y[i,j] <= y[i, j+1]); 

    v = []
    for i = 1:length(LTE.splits_ordered)
        push!(v, vcat(L_bounds[i], LTE.splits_ordered[i], U_bounds[i]));
    end

    # (2.2.6)
    @constraint(model, [i = 1:LTE.n_feats], x[i] >=  v[i][1] + sum((v[i][j] - v[i][j-1])*(1 - y[i, j-1]) for j = 2 : LTE.n_splits[i]+1));

    # (2.2.7)
    @constraint(model, [i = 1:LTE.n_feats], x[i] <=  v[i][LTE.n_splits[i]+2] + sum((v[i][j] - v[i][j+1])* y[i, j-1] for j = 2 : LTE.n_splits[i]+1));

    # (2.2.2)
    @constraint(model, [tree = 1:LTE.n_trees], sum(z[tree, i] for i=1:LTE.n_leaves[tree]) == 1);
    
    for tree in 1:LTE.n_trees
        for current_node in findall(s -> s==true, LTE.split_nodes[tree])

            right_leaves = LTE.child_leaves[tree][current_node << 1 + 1];
            left_leaves = LTE.child_leaves[tree][current_node << 1];

            current_feat, current_splitpoint_index = LTE.splits[tree, current_node];

            # (2.2.3)
            @constraint(model, sum(model[:z][tree, leaf] for leaf in right_leaves) <= 1 - model[:y][current_feat, current_splitpoint_index]);

            # (2.2.4)
            @constraint(model, sum(model[:z][tree, leaf] for leaf in left_leaves) <= model[:y][current_feat, current_splitpoint_index]);
        end
    end

    if bound_tightening=="quadratic"
        @variable(model, d);
        # (2.2.1)
        @constraint(model, d == sum((LTE.a[tree][LTE.leaves[tree][leaf], :]'*x+LTE.b[tree][LTE.leaves[tree][leaf]])*z[tree, leaf] for tree = 1:LTE.n_trees, leaf = 1:LTE.n_leaves[tree]));
    
    elseif  bound_tightening=="output"

        @variable(model, d[1:LTE.n_trees]);

        # It doesn't work
        # ---------------------
        
        @constraint(model, [tree = 1:LTE.n_trees], d_bounds[1]<=d[tree]<=d_bounds[2]);
        bounds = [[calculate_bounds(model, LTE, leaf, tree, d_bounds) for leaf in 1:LTE.n_leaves[tree]] for tree in 1:LTE.n_trees];
        # (2.2.12)
        @constraint(model, [tree = 2:LTE.n_trees, leaf = 1:LTE.n_leaves[tree]], LTE.a[tree][LTE.leaves[tree][leaf], :]'*x+LTE.b[tree][LTE.leaves[tree][leaf]] - d[tree] <=bounds[tree][leaf][1]*(1 - z[tree, leaf]));
        
        # (2.2.13)
        @constraint(model, [tree = 2:LTE.n_trees, leaf = 1:LTE.n_leaves[tree]], LTE.a[tree][LTE.leaves[tree][leaf], :]'*x+LTE.b[tree][LTE.leaves[tree][leaf]] - d[tree] >=-bounds[tree][leaf][2]*(1 - z[tree, leaf]));
        
        # ---------------------
    end 

     
   end

LTE_formulate! (generic function with 2 methods)

In [210]:
model[:d][1]

d[1]

In [211]:
model = Model(() -> Gurobi.Optimizer());

LTE_formulate!(model, LTE, U_bounds, L_bounds, bound_tightening = "output", d_bounds=d_bounds)

Set parameter Username
Academic license - for non-commercial use only - expires 2025-05-20


MethodError: MethodError: no method matching -(::Int64, ::Matrix{Float32})
For element-wise subtraction, use broadcasting with dot syntax: scalar .- array

Closest candidates are:
  -(::Real, !Matched::Complex{Bool})
   @ Base complex.jl:321
  -(!Matched::MutableArithmetics.Zero, ::Any)
   @ MutableArithmetics ~/.julia/packages/MutableArithmetics/NIXlP/src/rewrite.jl:63
  -(::Any, !Matched::MutableArithmetics.Zero)
   @ MutableArithmetics ~/.julia/packages/MutableArithmetics/NIXlP/src/rewrite.jl:64
  ...


In [206]:
@objective(model, Min, sum(model[:d]))

d[1] + d[2]

In [184]:
model

A JuMP Model
Minimization problem with:
Variables: 39
Objective function type: AffExpr
`AffExpr`-in-`MathOptInterface.EqualTo{Float64}`: 2 constraints
`AffExpr`-in-`MathOptInterface.GreaterThan{Float64}`: 10 constraints
`AffExpr`-in-`MathOptInterface.LessThan{Float64}`: 50 constraints
`QuadExpr`-in-`MathOptInterface.EqualTo{Float64}`: 1 constraint
`VariableRef`-in-`MathOptInterface.ZeroOne`: 32 constraints
Model mode: AUTOMATIC
CachingOptimizer state: ATTACHED_OPTIMIZER
Solver name: Gurobi
Names registered in the model: d, x, y, z

In [207]:
optimize!(model)

Gurobi Optimizer version 11.0.2 build v11.0.2rc0 (mac64[arm] - Darwin 23.4.0 23E224)

CPU model: Apple M1 Pro
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 94 rows, 39 columns and 245 nonzeros
Model fingerprint: 0x6446d518
Variable types: 7 continuous, 32 integer (32 binary)
Coefficient statistics:
  Matrix range     [1e-02, 1e+30]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [3e-01, 1e+30]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.

MIP start from previous solve did not produce a new incumbent solution

Presolve removed 16 rows and 0 columns
Presolve time: 0.00s

Explored 0 nodes (0 simplex iterations) in 0.00 seconds (0.00 work units)
Thread count was 1 (of 8 available processors)

Solution count 0

Model is infeasible or unbounded
Best objective -, best bound -, gap -

User-callback calls 37, time in user-callback 0.00 sec


In [208]:
objective_value(model)

MathOptInterface.ResultIndexBoundsError{MathOptInterface.ObjectiveValue}: Result index of attribute MathOptInterface.ObjectiveValue(1) out of bounds. There are currently 0 solution(s) in the model.

In [187]:
EvoTrees.predict(evo_model, value.(model[:x])')[1] 

-0.016500596f0